# <font color='Red'>DDB (DESHIMA DATABASE)</font>

## Import Modules

In [10]:
import os
from pathlib import Path

import yaml
import numpy as np
from datetime import datetime
from astropy.io import fits
from astropy.io import ascii
from astropy import table
from astropy import coordinates
from astropy import units as u

In [11]:
D_ASTE    = (10.0* u.m).value                            # diameter of the ASTE
LON_ASTE  = coordinates.Angle('-67d42m11.89525s').deg    # longitude of the ASTE
LAT_ASTE  = coordinates.Angle('-22d58m17.69447s').deg    # latitude of the ASTE
PATH_DDBDICT = Path('~/DESHIMA/datafile/DDB_dict.yaml').expanduser()
FORM_FITSTIME  = '%Y-%m-%dT%H:%M:%S'

In [12]:
def convert_fitstime(time_lis):
    if len(time_lis[0]) <= 14:
        return [datetime.strftime(datetime.strptime(time_lis[i], '%Y%m%d%H%M%S'),
                                  FORM_FITSTIME)
                for i in range(len(time_lis))]
    else:
        return [datetime.strftime(datetime.strptime(time_lis[i], '%Y%m%d%H%M%S.%f'),
                                  FORM_FITSTIME + '.%f')
                for i in range(len(time_lis))]
    
def addHeaderComments(hdr, key, com):
    hdr.comments[key] = com


def createBinTableHdu(data_dict):
    header = fits.Header()
    for (i, j, k) in zip(data_dict['hdr_key_lis'], data_dict['hdr_val_lis'], data_dict['hdr_com_lis']):
        header[i] = j, k

    columns = [fits.Column(name=data_dict['cols_key_lis'][i],
                           format=data_dict['tform'][i],
                           array=data_dict['cols_data_lis'][i],
                           unit=data_dict['tunit'][i])
               for i in range(len(data_dict['cols_key_lis']))]

    hdu = fits.BinTableHDU.from_columns(columns, header)
    [addHeaderComments(hdu.header, list(hdu.header)[i], data_dict['hdr_com_lis'][i])
     for i in range(- (len(data_dict['hdr_com_lis']) - data_dict['hdr_com_lis'].index('label for field 1')), 0)]

    return hdu

In [13]:
 def make_kidsinfo(kidsinfo_data):
    filters_data = Path(kidsinfo_data).expanduser()
    data = np.load(filters_data)

    pixelid   = np.zeros(49)
    kidid     = data['kidid']
    kidtypes  = np.append(np.append(np.zeros(44), np.ones(4)), np.ones(1)* 2)
    bandpass  = data['bandpass']
    lorentz   = data['lorentz']


    tform4 = str(len(bandpass[0])) + 'D'
    
    filename = os.path.basename(filters_data)
    crval3   = 325.0e+09
    crpix3   = 0.0
    cdelt3   = 10.0e+06

    with open(PATH_DDBDICT, 'r') as f:
        ddb_dict = yaml.load(f)

    kidsinfo_dict = ddb_dict['kidsinfo_dict']

    kidsinfo_dict['hdr_val_lis'][1] = filename
    kidsinfo_dict['hdr_val_lis'][6:] = [crval3, crpix3, cdelt3]
    kidsinfo_dict['cols_data_lis'] = [pixelid, kidid, kidtypes, bandpass, lorentz]
    kidsinfo_dict['tform'][3] = tform4
    
    return createBinTableHdu(kidsinfo_dict)

In [14]:
 def make_rcp():
#    filters_data = Path(rcp_data).expanduser()

    pixelid   = np.arange(0, 1)
    offsetaz  = np.zeros(1)
    offsetel  = np.zeros(1)
    gain      = np.ones(1)
    
    filename = None

    with open(PATH_DDBDICT, 'r') as f:
        ddb_dict = yaml.load(f)

    rcp_dict = ddb_dict['rcp_dict']

    rcp_dict['hdr_val_lis'][1] = filename
    rcp_dict['cols_data_lis'] = [pixelid, offsetaz, offsetel, gain]

    return createBinTableHdu(rcp_dict)

In [15]:
hdus = fits.HDUList()
hdus.append(fits.PrimaryHDU())
hdus['PRIMARY'].header['DDB_ID'] = '20171018151938', 'The ID of DESHIMA DataBase'
hdus.append(make_kidsinfo('~/DESHIMA/datafile/Filterbank/filterbank_model.npz'))
hdus.append(make_rcp())

In [16]:
hdus.info()

Filename: (No file associated with this HDUList)
No.    Name         Type      Cards   Dimensions   Format
  0  PRIMARY     PrimaryHDU       5   ()      
  1  KIDSINFO    BinTableHDU     29   49R x 5C   ['K', 'K', 'K', '400D', '4D']   
  2  RCP         BinTableHDU     21   1R x 4C   ['K', 'D', 'D', 'D']   


In [17]:
table.Table(hdus['KIDSINFO'].data)

pixelid,kidid,kidtypes,bandpass [400],lorentz [4]
int64,int64,int64,float64,float64
0,0,0,0.0461851887557 .. 0.037740299465,0.0 .. 0.0
0,1,0,0.0483083776886 .. 0.0469188614852,0.0 .. 0.0
0,2,0,0.0246086646002 .. 0.0372831801211,0.0 .. 0.0
0,3,0,0.0362856914045 .. 0.0301401109486,0.0 .. 0.0
0,4,0,0.0462420745253 .. 0.0548321279313,0.0 .. 0.0
0,5,0,0.0415924244039 .. 0.0578518133164,0.0 .. 0.0
0,6,0,0.0515102134994 .. 0.0381329095985,0.0 .. 0.0
0,7,0,0.048596208546 .. 0.0483828067362,0.0 .. 0.0
0,8,0,0.0471632282672 .. 0.0605464043199,0.0 .. 0.0


In [18]:
ddb_fitsname = 'DDB_' + hdus['PRIMARY'].header['DDB_ID'] + '.fits'
hdus.writeto('/Users/tetsu/DESHIMA/datafile/' + ddb_fitsname)